# CAR-19 Data Analysis Goal 2: Confirm Subarray Position

This notebook runs the code in the nircam_calib/comissioning/NRC_19_subarrays module in order to analyze data from CAR-19 (Subarray Verification)

## Table of Contents

* [Goal 2: Confirm positions of subarrays on detectors](#goal_2)

In [ ]:
from glob import glob
import os

In [ ]:
from astropy.io import ascii, fits
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.pipeline.calwebb_image2 import Image2Pipeline
from jwst.pipeline.calwebb_image3 import Image3Pipeline
from jwst.pipeline.calwebb_tso3 import Tso3Pipeline
from jwst.source_catalog import SourceCatalogStep
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from nircam_calib.commissioning.NRC_19_subarrays.subarray_loc_examination_prep import check_location
from nircam_calib.commissioning.NRC_19_subarrays import confirm_subarray_location_via_sources as locations
from nircam_calib.commissioning.NRC_19_subarrays import confirm_telescope_pointing as pointing
from nircam_calib.commissioning.NRC_19_subarrays import confirm_count_rates as count_rates
from nircam_calib.commissioning.utils.display import overlay_catalog

In [ ]:
import jwst
print(jwst.__version__)

In [ ]:
base_dir = '/ifs/jwst/wit/nircam/simulationWG/Imaging/CAR-19/2021_June'
#base_dir = 'path_to_simulationWG_CAR-19_directory'

In [ ]:
def display(files, titles, minval=0, maxval=10):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 10))

    for ax, filename, title in zip(axes.flat, files, titles):
        img = fits.getdata(filename)
        ax.set_axis_off()
        im = ax.imshow(img, origin='lower', cmap='viridis', vmin=minval, vmax=maxval)
        ax.set_title(title)

    cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.5)
    cbar.set_ticks(np.linspace(minval, maxval, 5))
    plt.show()

<a id='goal_2'></a>
## Goal 2: Confirm positions of subarrays on detectors are correct

Make sure that the input subarrays have fields of view that overlap the full frame file!!

### Primary check: 

Using the SUBSTRT1 and SUBSTRT2 header keyword values in the subarray files, extract a subarray from an accompanying full frame exposure and see if the bias structure/visible hot/dead pixels are in the same locations of each.

In [ ]:
# Uncal files to use for visual subarray location check. Include all point
# source and extended subarrays. Note that we can't do this for the 
# TSO observation because we have no full frame A1, A3, or A5 exposure to
# compare to. We'll have to grab full frame A module exposures from some
# other program. See the cell below for the placeholder entries for the A module.
full_uncal_files = {'B1': 'Mirage_Output/jw01068004001_01101_00001_nrcb1_uncal.fits',
                    'B2': 'Mirage_Output/jw01068004001_01101_00001_nrcb2_uncal.fits',
                    'B3': 'Mirage_Output/jw01068004001_01101_00001_nrcb3_uncal.fits',
                    'B4': 'Mirage_Output/jw01068004001_01101_00001_nrcb4_uncal.fits',
                    'B5': 'Mirage_Output/jw01068004001_01101_00001_nrcb5_uncal.fits',
                    }

subarray_uncal_files = {'B1': ['Mirage_Output/jw01068001001_01101_00001_nrcb1_uncal.fits',
                               'Mirage_Output/jw01068002001_01101_00001_nrcb1_uncal.fits',
                               'Mirage_Output/jw01068003001_01101_00001_nrcb1_uncal.fits',
                               'Mirage_Output/jw01068005001_01101_00001_nrcb1_uncal.fits',
                               'Mirage_Output/jw01068006001_01101_00001_nrcb1_uncal.fits'],
                        'B2': ['Mirage_Output/jw01068001001_01101_00001_nrcb2_uncal.fits',
                               'Mirage_Output/jw01068002001_01101_00001_nrcb2_uncal.fits',
                               'Mirage_Output/jw01068003001_01101_00001_nrcb2_uncal.fits'],
                        'B3': ['Mirage_Output/jw01068001001_01101_00001_nrcb3_uncal.fits',
                               'Mirage_Output/jw01068002001_01101_00001_nrcb3_uncal.fits',
                               'Mirage_Output/jw01068003001_01101_00001_nrcb3_uncal.fits'],
                        'B4': ['Mirage_Output/jw01068001001_01101_00001_nrcb4_uncal.fits',
                               'Mirage_Output/jw01068002001_01101_00001_nrcb4_uncal.fits',
                               'Mirage_Output/jw01068003001_01101_00001_nrcb4_uncal.fits'],
                        'B5': ['Mirage_Output/jw01068001001_01101_00001_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068002001_01101_00001_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068003001_01101_00001_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068005001_01101_00001_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068005001_01101_00002_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068006001_01101_00001_nrcb5_uncal.fits',
                               'Mirage_Output/jw01068006001_01101_00002_nrcb5_uncal.fits'],
                       }

In [ ]:
for detector in full_uncal_files:
    for subfile in subarray_uncal_files[detector]:
        check_location(full_uncal_files[detector], subfile)
        
# Resulting files have two extensions: 1 for the true subarray and 1 for the subarray
# extracted from the full frame. Easiest check is to blink these in ds9.

## Now open the files listed above in ds9 and compare subarray observations with extracted subarrays from full frame observations. Compare locations of bias structures and bad pixels.

In [ ]:
# Placeholder for the same check as above, but using the A module detectors
# in order to compare the TSO observation subarrays (including TA exposure)
# Full frame exposures will have to come from some other program
full_uncal_files = {'A1': 'xxxxxx_nrca1_uncal.fits',
                    'A3': 'xxxxxx_nrca3_uncal.fits',
                    'A5': 'xxxxxx_nrca5_uncal.fits',
                    }

subarray_uncal_files = {'A1': ['Mirage_Output/jw01068007001_01101_00001-seg001_nrca1_uncal.fits'],
                        'A3': ['Mirage_Output/jw01068007001_01101_00001-seg001_nrca3_uncal.fits'],
                        'A5': ['Mirage_Output/jw01068007001_01101_00001_nrca5_uncal.fits',
                               'Mirage_Output/jw01068007001_01101_00002-seg001_nrca5_uncal.fits']
                       }


### Secondary check:

<b>This should be considered a secondary check compared to the manual extraction and examination above, since we may not have a valid WCS at the time the data are taken.</b>

Locate sources on the full frame and subarray exposures and compare vs expectations. 

In [ ]:
# B1 - SUB160, SUB320, SUB640, FULL
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb1_cal.fits'
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb1_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B2 - SUB160, SUB320, SUB640, FULL
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb2_cal.fits'
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb2_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb2_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb2_cal.fits']

#subs = ['Pipeline_Level2/jw01068002001_01101_00001_nrcb2_cal.fits',
#        'Pipeline_Level2/jw01068003001_01101_00001_nrcb2_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B3 - SUB160, SUB320, SUB640, FULL
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb3_cal.fits'
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb3_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb3_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb3_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B4 - SUB160, SUB320, SUB640, FULL
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb4_cal.fits'
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb4_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb4_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb4_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B5 - SUB160, SUB320, SUB640, FULL
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits'
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb5_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B1 - SUB64P, SUB600P
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb1_cal.fits'
subs = ['Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)

In [ ]:
# B5 - SUB64P, SUB600P
full = 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits'
subs = ['Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits']
locations.run(full, subs, fullframe_threshold=500, subarray_threshold=500)